In [1]:
# objective is to make a many-to-many table
# requires roster json file

import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite') # creates this file if does not exist
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER PRIMARY KEY,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER PRIMARY KEY,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if len(fname) < 1:
    fname = 'roster_data_sample.json'

#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

# extract data from json file and add to database 
for entry in json_data:

    name = entry[0]
    title = entry[1]
    role = entry[2]

    print((name, title, role))

# OR IGNORE means if the insert causes it to blowup, ignore data
    # because of inserting a replicate into a unique constraint 

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) ) 

# inserts name into User.name, ? used as placeholder (avoids sql injection)
    # auto makes id which we need to retrieve:
    
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0] # if there was more than one thing [0] means first thing is name

# same logic above for getting course title and retrieivng id
    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]


# I manually added roles into the member table luckily json is organised
    # otherwise would need to add member based on user_id and course_id from the other file
    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role ) 
                VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role ) )

# if (user_id, course_id) primary key is already there, update it
    # have (?, ?) so need 2 tuples for python 

    conn.commit() # writes into disk but has to pause until done (sometimes not run every time)